# cheerio / axios를 이용해서 웹크롤링을 해보자~
SYNOPSIS<br>
    [.도움 (.help)]<br>
    [.뉴스 (.news)]<br>
<br>
DESCRIPTION<br>
    .help: 도움말<br>
    .news: 최신보안뉴스<br>
<br>
EXTENSION<br>
    [.실검 (.realtime)]<br>
    [.날씨 지역 (.weather local)]<br>
    [.환율 (.exchange)]<br>
    [.운세 (.luck)]<br>
    [.운세 띠 (.luck zodiac_name)]<br>
    [.로또 (.lotto)]<br>
    [.음식메뉴 (.foodmenu)]<br>
    [.소라고동 내용 (.conch text)]<br>
    [.노래추천 (.music)]<br>


## 네이버 뉴스 취득
* 이런식으로 패키지를 불러드릴 수 있다.
* 아래의 라이브러리가 안 깔려 있을 경우에 [npm install axios]와 같이 명령어를 실행해준다.
* 처음 시도할 시에는 [npm init] 명령어로 프로젝트를 작성할 필요가 있음..ㅎㅎ

In [57]:
var axios    = require('axios')
var cheerio  = require('cheerio')
var iconv    = require('iconv-lite')
var log      = console.log

###### 콜백함수로 코드를 좀 줄여봅시다..

In [141]:
async function execAxios(obj) {
    obj.url      = obj.url      || ""
    obj.subUrl   = obj.subUrl   || ""
    obj.callback = obj.callback || function(){}
    var url   = obj.url
    var NAVER = obj.subUrl
    var data  = ""
    try {
        data = await axios.get(url)
        data = data.data
    } catch (err) {
        console.log(err)
    }
    obj.callback(data)
}
execAxios({
    url     : "https://news.naver.com/main/ranking/popularLike.nhn?subType=20",
    subUrl  : "https://news.naver.com",
    callback: function(data){
        
    const $         = cheerio.load(data, {decodeEntities: false});
    const $bodyList = $("div.likeitnews_item_text")
    var   str       = ""
    
    $bodyList.each(function(i) {
    str += `
${$(this).find('a.likeitnews_item_title').html()}
<br>
${$(this).find('div.likeitnews_item_lede').html()}
<br>
${this.subUrl + $(this).find('a.likeitnews_item_title').attr("href")}
<br>
<br>
`
    if ( i == 0) console.log(str)
    })
    }
})


[�ܵ�] '��������' ���� ���ʷ� ��������顤���ְ� ������ ���� ��
<br>
�޹� 1�� ������ 1600���̴�. 6��¥�� ���� ��ǰ�� 7280���� �ȸ���. ���� ��ǰ�� ���� ������ ���� ��ǰ���� 25% ���� �δ�. ������ 1�Ϻ��� ������Ʈ��
<br>
undefined/main/ranking/read.nhn?rankingType=popular_like&oid=015&aid=0004364124&date=&type=1&rankingSectionId=001&rankingSeq=1
<br>
<br>



### 아래의 URL의 사이트에서 20대에게 인기있는 10가지 기사를 취득해보자
* URL : https://news.naver.com/main/ranking/popularLike.nhn?subType=20



In [139]:
async function getNews() {
    var url   = 'https://news.naver.com/main/ranking/popularLike.nhn?subType=20'
    var NAVER = "https://news.naver.com"
    var str   = ""
    var data  = ""
    try {
        data = await axios.get(url)
        data = data.data
    } catch (err) {
        console.log(err)
    }
    const $ = cheerio.load(data, {decodeEntities: false});
    const $bodyList = $("div.likeitnews_item_text")
    $bodyList.each(function(i) {
    str += `
${$(this).find('a.likeitnews_item_title').html()}
<br>
${$(this).find('div.likeitnews_item_lede').html()}
<br>
${NAVER + $(this).find('a.likeitnews_item_title').attr("href")}
<br>
<br>
`
    if ( i == 0) console.log(str)
    })
}
getNews()


[�ܵ�] '��������' ���� ���ʷ� ��������顤���ְ� ������ ���� ��
<br>
�޹� 1�� ������ 1600���̴�. 6��¥�� ���� ��ǰ�� 7280���� �ȸ���. ���� ��ǰ�� ���� ������ ���� ��ǰ���� 25% ���� �δ�. ������ 1�Ϻ��� ������Ʈ��
<br>
https://news.naver.com/main/ranking/read.nhn?rankingType=popular_like&oid=015&aid=0004364124&date=&type=1&rankingSectionId=001&rankingSeq=1
<br>
<br>



##### 아니.. 성공했다고 생각했더니 위와 같은 문자깨짐이 일어난다..;

아마도 문제는
euc-kr -> UTF-8로 변환을 하고 있기 때문일 것으로 추측..]
axios로 데이터를 취득 시 변환작업을 안하도록 설정을 해본다.



##### 이번에 접근할 방법은 iconv-lite로 [arrayBuffer]에서 [EUC-KR]로 취득하는 방법이 되겠다 ㅎㅎ

적용방법 : 
1. var iconv    = require('iconv-lite')
2. axios의 [responseType -> arraybuffer]
3. iconv.decode("인코딩을 변경할 문자열", "EUC-KR").toString()

In [106]:
async function getNews() {
    var url   = 'https://news.naver.com/main/ranking/popularLike.nhn?subType=20'
    var NAVER = "https://news.naver.com"
    var str   = ""
    var data  = ""
    try {
        data = await axios.get(url, {method: "GET", responseType: "arraybuffer" })
        data = data.data
        data = iconv.decode(data, "EUC-KR").toString()
    } catch (err) {
        console.log(err)
    }
    const $ = cheerio.load(data, {decodeEntities: false});
    const $bodyList = $("div.likeitnews_item_text")
    $bodyList.each(function(i) {
    str += `
${$(this).find('a.likeitnews_item_title').html()}
<br>
${$(this).find('div.likeitnews_item_lede').html()}
<br>
${NAVER + $(this).find('a.likeitnews_item_title').attr("href")}
<br>
<br>
`
    if ( i == 0) console.log(str)
    })
}
getNews()


3040 부글부글 "흙수저는 평생 전세로만 살라는거냐"
<br>
정부 부동산 대책에 와글와글 "현금 부자들만 더 부자로 만드는 정책" 전문가 "최대 피해자는 40대 이하 무주택자" “편도 1시간 넘는 콩나물 열차를 타…
<br>
https://news.naver.com/main/ranking/read.nhn?rankingType=popular_like&oid=023&aid=0003540038&date=&type=0&rankingSectionId=001&rankingSeq=1
<br>
<br>



## 인크루트 채용공고 취득 ㅎㅎ

In [113]:

async function getJobList() {
    var url = "http://job.incruit.com/jobdb_list/searchjob.asp?ct=12&ty=1&cd=1&occ2=574&jobty=1,2&page=1&apply=m#applytopcomp"
    var str   = ""
    var data  = ""
    try {
        data = await axios.get(url, {method: "GET", responseType: "arraybuffer" })
        data = data.data
        data = iconv.decode(data, "EUC-KR").toString()
    } catch (err) {
        console.log(err)
    }
    const $ = cheerio.load(data, {decodeEntities: false});
    const $bodyList = $("div.n_job_list_table_a.list_full_default tr")
    $bodyList.each(function(i) {
        let company = $(this).find('.companys.check_companys > .check_list_r > .links > a') || $("")
        let title   = $(this).find('div.subjects > span.accent > a.links') || $("")
        let target  = $(this).find('div.subjects > p.details_txts > em').eq(1)   || $("")

        company     = (company.html() || "").trim()
        link        = (title.attr("href") || "").trim()
        title       = (title.html()   || "").trim()
        target      = (target.html()  || "").trim().replace("\<span>", "").replace("\</span>", "")
        if (company == "") return
    str += `
${i}. ${company}
<br>
${title}
<br>
${target}
<br>
${link}
<br>
<br>
`
    if ( i < 2) console.log(str)
    })
    //console.log("<br>** 바로지원 쌉가능 ~ 지원공고 입니다 냥냥 **<br>" + str)
}
getJobList()


1. (주)호디
<br>
JSP, Servlet 외 신입/경력 정규직, 병역특례 채용
<br>
신입/경력1년↑|학력무관
<br>
http://job.incruit.com/jobdb_info/jobpost.asp?job=2006110001008
<br>
<br>



In [131]:


async function gethotIssueList() {
    var url = "https://datalab.naver.com/keyword/realtimeList.naver?age=20s&groupingLevel=4&marketing=-1&where=main"
    var str   = ""
    var data  = ""
    try {
        data = await axios.get(url, {method: "GET", responseType: "arraybuffer" })
        data = data.data
        //data = iconv.decode(data, "EUC-KR").toString()
    } catch (err) {
        console.log(err)
    }
    const $ = cheerio.load(data, {decodeEntities: false});
    const $bodyList = $("div.ranking_box .item_title")
    $bodyList.each(function(i) {
        let title = $(this) || $("")
        title     = (title.html() || "").trim()
        if (title == "") {
            return
        }
    str += `
${i + 1}. ${title}
<br>
`
    })
    console.log("<br>** 네이버 급상승 검색어 입니다 냥냥 **<br>" + str)
}
gethotIssueList()

<br>** 네이버 급상승 검색어 입니다 냥냥 **<br>
1. 이강인
<br>

2. 큰고니
<br>

3. 굿걸
<br>

4. 레알마드리드
<br>

5. 로드 투 킹덤
<br>

6. 블루
<br>

7. 토트넘 맨유
<br>

8. 안소현
<br>

9. lck
<br>

10. 사라진 시간
<br>

11. 우지윤
<br>

12. 외모지상주의
<br>

13. 오늘부산날씨
<br>

14. 노수광
<br>

15. 장예은
<br>

16. 전지우
<br>

17. 현대엘리베이터
<br>

18. 이태양
<br>

19. 유현주
<br>

20. 두근두근두근거려
<br>



## 날씨 취득 ~ 

In [233]:
execAxios({
    url     : "https://weather.naver.com/rgn/cityWetrMain.nhn",
    subUrl  : "",
    callback: function(data){
        //console.log(data)
        const $       = cheerio.load(data, {decodeEntities: false});
        var str       = ""
        var $bodyList = $(".tbl_weather.tbl_today tr")
        $bodyList     = $bodyList.eq(1)
        $bodyList     = $bodyList.find("td")
        $bodyList.each(function(i){
            let nalsee = $(this).find("ul.text li.nm")
            let temp   = $(this).find("ul.text li span.temp")
            let rain   = $(this).find("ul.text li span.rain")

            nalsee     = nalsee.html()

            temp       = temp.html()
            temp       = temp.replace("\<strong>", "")
            temp       = temp.replace("\</strong>", "")

            rain       = rain.html().replace("\<strong>", "")
            rain       = rain.replace("\</strong>", "")
            if(i == 0) {
                status = "오전"
            } else {
                status = "오후"
            }
            
            str += `
${new Date().toLocaleDateString()} ${status}기준 <br>
${nalsee} 온도[${temp}] 습도[${rain}]<br>
    `
            console.log(str)
    })
    }
})


2020-6-20 오전기준 <br>
맑음 온도[21.0℃] 습도[10%]<br>
    

2020-6-20 오전기준 <br>
맑음 온도[21.0℃] 습도[10%]<br>
    
2020-6-20 오후기준 <br>
맑음 온도[30.0℃] 습도[0%]<br>
    


In [231]:

execAxios({
    url     : "https://weather.naver.com/period/weeklyFcast.nhn",
    subUrl  : "",
    callback: function(data){
        // console.log(data)
        const $       = cheerio.load(data, {decodeEntities: false});
        var str       = ""
        var $bodyList = $(".tbl_type5.tbl_wk thead th[colspan]")
        var weatherDetailList = $(".tbl_wk tbody tr").eq(1).find("td img")
        $bodyList.each(function(i){
            var temp = ""
            //let nalsee = $(this).find("ul.text li.nm")
            //let temp   = $(this).find("ul.text li span.temp")
            //let rain   = $(this).find("ul.text li span.rain")
            var self = this
            
            $([1,2]).each(function(){
                if (this == 1) {
                    temp += `
${$(self).html().trim() + " 오전 " + weatherDetailList.eq((i) * 2).attr("title")}
`
                } else {
                    temp += `
${$(self).html().trim() + " 오후 " + weatherDetailList.eq(i * 2 + 1).attr("title")}
`
                }
                temp += "<br>"
            })
            str = temp
 
            console.log(str)
    })
    }
})

/*
https://weather.naver.com/period/weeklyFcast.nhn
var weatherDetailList = $(".tbl_type5.tbl_wk tbody tr").eq(1).find("td img")
$(".tbl_type5.tbl_wk thead th[colspan]").each(function(i){
    var self = this
    $([1,2]).each(function(){
        if (this == 1) {
            console.log($(self).html().trim(), "오전", weatherDetailList.eq((i) * 2).attr("title"))

        } else {
            console.log($(self).html().trim(), "오후", weatherDetailList.eq(i * 2 + 1).attr("title"))
        }
    })
})
*/


6.21. (일) 오전 맑음
<br>
6.21. (일) 오후 맑음
<br>

6.22. (월) 오전 맑음
<br>
6.22. (월) 오후 맑음
<br>

6.23. (화) 오전 구름많음
<br>
6.23. (화) 오후 구름많음
<br>

6.24. (수) 오전 구름많음
<br>
6.24. (수) 오후 흐리고 비
<br>

6.25. (목) 오전 흐리고 비
<br>
6.25. (목) 오후 흐리고 비
<br>

6.26. (금) 오전 흐리고 비
<br>
6.26. (금) 오후 구름많음
<br>
